In [1]:
import azureml.core
from azureml.core import Workspace
import os
from azureml.core import Run
import pandas as pd
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
import statistics
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import os
import time
import warnings
import gc
gc.collect()
import os
# from six.moves import urllib
import matplotlib
import matplotlib.pyplot as plt
# import seaborn as sns
import datetime
warnings.filterwarnings('ignore')
%matplotlib inline
# plt.style.use('seaborn')
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler

In [3]:
#Add All the Models Libraries

# Scalers
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

# Models

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error

from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from scipy.stats import reciprocal, uniform

from sklearn.model_selection import StratifiedKFold, RepeatedKFold

# Cross-validation
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import cross_validate

# GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Common data processors
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from scipy import sparse

In [4]:
ws = Workspace.from_config()

In [5]:
run = Run.get_context()

In [6]:
# to make this notebook's output stable across runs
np.random.seed(123)
gc.collect()
# To plot pretty figures
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [11]:
#Reduce the memory usage - Inspired by Panchajanya Banerjee
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [12]:
def one_hot_encoder(df, nan_as_category=True):
    original_columns = df.columns.tolist()

    categorical_columns = list(filter(lambda c: c in ['object'], df.dtypes))
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)

    new_columns = list(filter(lambda c: c not in original_columns, df.columns))
    return df, new_columns

In [13]:
train = reduce_mem_usage(pd.read_csv('azureml://subscriptions/a15355bc-32e5-4151-bb4c-a2ecff532aa1/resourcegroups/ml/workspaces/MLL/datastores/workspaceblobstore/paths/UI/2024-03-24_084703_UTC/train.csv',parse_dates=["first_active_month"]))
test = reduce_mem_usage(pd.read_csv('azureml://subscriptions/a15355bc-32e5-4151-bb4c-a2ecff532aa1/resourcegroups/ML/workspaces/MLL/datastores/workspaceblobstore/paths/UI/2024-03-20_141609_UTC/test.csv', parse_dates=["first_active_month"]))

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)


In [15]:
train.shape
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820312
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392822
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.687988
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142456
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159790
...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,-2.740234
201913,2015-10-01,C_ID_1314773c0b,3,1,1,0.312988
201914,2017-08-01,C_ID_7666735b3d,4,3,0,0.093506
201915,2016-07-01,C_ID_73f5a0efd0,3,2,1,-4.675781


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   first_active_month  201917 non-null  datetime64[ns]
 1   card_id             201917 non-null  object        
 2   feature_1           201917 non-null  float64       
 3   feature_2           201917 non-null  float64       
 4   feature_3           201917 non-null  float64       
 5   target              201917 non-null  float16       
 6   outliers            201917 non-null  int64         
dtypes: datetime64[ns](1), float16(1), float64(3), int64(1), object(1)
memory usage: 9.6+ MB


In [17]:
# Remove the Outliers if any 
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

for features in ['feature_1','feature_2','feature_3']:
    order_label = train.groupby([features])['outliers'].mean()
    train[features] = train[features].map(order_label)
    test[features] =  test[features].map(order_label)

In [53]:
# Now extract the days and Qtr

train['days'] = (datetime.datetime(2018,2,1) - (train['first_active_month'].dt.date).apply(pd.Timestamp)).dt.days
train['quarter'] = train['first_active_month'].dt.quarter

feature_cols = ['feature_1', 'feature_2', 'feature_3']
for f in feature_cols:
    train['days_' + f] = train['days'] * train[f]
    train['days_' + f + '_ratio'] = train[f] / train['days']

test['days'] = (datetime.datetime(2018,2,1) - (test['first_active_month'].dt.date).apply(pd.Timestamp)).dt.days
test['quarter'] = test['first_active_month'].dt.quarter

feature_cols = ['feature_1', 'feature_2', 'feature_3']
for f in feature_cols:
    test['days_' + f] = test['days'] * test[f]
    test['days_' + f + '_ratio'] = test[f] / test['days']

gc.collect()

21

In [55]:
def aggregate_transaction_hist(trans, prefix):  
        
    agg_func = {
        'purchase_amount' : ['sum','max','min','mean','var','skew'],
        'installments' : ['sum','max','mean','var','skew'],
        'purchase_date' : ['max','min'],
        'month_lag' : ['max','min','mean','var','skew'],
        'month_diff' : ['max','min','mean','var','skew'],
        'weekend' : ['sum', 'mean'],
        'weekday' : ['sum', 'mean'],
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum','mean', 'max','min'],
        'card_id' : ['size','count'],
        'month': ['nunique', 'mean', 'min', 'max'],
        'hour': ['nunique', 'mean', 'min', 'max'],
        'weekofyear': ['nunique', 'mean', 'min', 'max'],
        'day': ['nunique', 'mean', 'min', 'max'],
        'subsector_id': ['nunique'],
        'merchant_category_id' : ['nunique'],
        'price' :['sum','mean','max','min','var'],
        'duration' : ['mean','min','max','var','skew'],
        'amount_month_ratio':['mean','min','max','var','skew']
        
    }
    
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() 
                           for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    
    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    
    return agg_trans

In [56]:
transactions = reduce_mem_usage(pd.read_csv('azureml://subscriptions/a15355bc-32e5-4151-bb4c-a2ecff532aa1/resourcegroups/ML/workspaces/MLL/datastores/workspaceblobstore/paths/UI/2024-03-24_085850_UTC/historical_transactions.csv'))
gc.collect()

Mem. usage decreased to 1749.11 Mb (43.7% reduction)


1645

In [60]:
#impute missing values - This is now excluded.
transactions['category_2'] = transactions['category_2'].fillna(1.0,inplace=True)
transactions['category_3'] = transactions['category_3'].fillna('A',inplace=True)
transactions['merchant_id'] = transactions['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
transactions['installments'].replace(-1, np.nan,inplace=True)
transactions['installments'].replace(999, np.nan,inplace=True)
transactions['purchase_amount'] = transactions['purchase_amount'].apply(lambda x: min(x, 0.8))

#Feature Engineering - Adding new features inspired by Chau's first kernel
transactions['authorized_flag'] = transactions['authorized_flag'].map({'Y': 1, 'N': 0})
transactions['category_1'] = transactions['category_1'].map({'Y': 1, 'N': 0})
transactions['category_3'] = transactions['category_3'].map({'A':0, 'B':1, 'C':2})

transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])
transactions['weekofyear'] = transactions['purchase_date'].dt.isocalendar().week
transactions['month'] = transactions['purchase_date'].dt.month
transactions['day'] = transactions['purchase_date'].dt.day
transactions['weekday'] = transactions.purchase_date.dt.weekday
transactions['weekend'] = (transactions.purchase_date.dt.weekday >=5).astype(int)
transactions['hour'] = transactions['purchase_date'].dt.hour 
transactions['month_diff'] = ((datetime.datetime.today() - transactions['purchase_date']).dt.days)//30
transactions['month_diff'] += transactions['month_lag']

# additional features
transactions['duration'] = transactions['purchase_amount']*transactions['month_diff']
transactions['amount_month_ratio'] = transactions['purchase_amount']/transactions['month_diff']
transactions['price'] = transactions['purchase_amount'] / transactions['installments']

gc.collect()

38770

In [61]:
agg_func = {
        'mean': ['mean'],
    }
for col in ['category_2','category_3']:
    transactions[col+'_mean'] = transactions['purchase_amount'].groupby(transactions[col]).agg('mean')
    transactions[col+'_max'] = transactions['purchase_amount'].groupby(transactions[col]).agg('max')
    transactions[col+'_min'] = transactions['purchase_amount'].groupby(transactions[col]).agg('min')
    transactions[col+'_sum'] = transactions['purchase_amount'].groupby(transactions[col]).agg('sum')
    agg_func[col+'_mean'] = ['mean']
    
gc.collect()

0

In [62]:
merge_trans = aggregate_transaction_hist(transactions, prefix='hist_')
del transactions
gc.collect()
train = pd.merge(train, merge_trans, on='card_id',how='left')
test = pd.merge(test, merge_trans, on='card_id',how='left')
del merge_trans
gc.collect()

32

In [64]:
#Feature Engineering - Adding new features inspired by Chau's first kernel
train['hist_purchase_date_max'] = pd.to_datetime(train['hist_purchase_date_max'])
train['hist_purchase_date_min'] = pd.to_datetime(train['hist_purchase_date_min'])
train['hist_purchase_date_diff'] = (train['hist_purchase_date_max'] - train['hist_purchase_date_min']).dt.days
train['hist_purchase_date_average'] = train['hist_purchase_date_diff']/train['hist_card_id_size']
train['hist_purchase_date_uptonow'] = (datetime.datetime.today() - train['hist_purchase_date_max']).dt.days
train['hist_purchase_date_uptomin'] = (datetime.datetime.today() - train['hist_purchase_date_min']).dt.days
train['hist_first_buy'] = (train['hist_purchase_date_min'] - train['first_active_month']).dt.days
train['hist_last_buy'] = (train['hist_purchase_date_max'] - train['first_active_month']).dt.days

for feature in ['hist_purchase_date_max','hist_purchase_date_min']:
    train[feature] = train[feature].astype(np.int64) * 1e-9
gc.collect()

547

In [65]:
#Feature Engineering - Adding new features inspired by Chau's first kernel
test['hist_purchase_date_max'] = pd.to_datetime(test['hist_purchase_date_max'])
test['hist_purchase_date_min'] = pd.to_datetime(test['hist_purchase_date_min'])
test['hist_purchase_date_diff'] = (test['hist_purchase_date_max'] - test['hist_purchase_date_min']).dt.days
test['hist_purchase_date_average'] = test['hist_purchase_date_diff']/test['hist_card_id_size']
test['hist_purchase_date_uptonow'] = (datetime.datetime.today() - test['hist_purchase_date_max']).dt.days
test['hist_purchase_date_uptomin'] = (datetime.datetime.today() - test['hist_purchase_date_min']).dt.days

test['hist_first_buy'] = (test['hist_purchase_date_min'] - test['first_active_month']).dt.days
test['hist_last_buy'] = (test['hist_purchase_date_max'] - test['first_active_month']).dt.days

for feature in ['hist_purchase_date_max','hist_purchase_date_min']:
    test[feature] = test[feature].astype(np.int64) * 1e-9

gc.collect()

105

In [66]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,days,quarter,days_feature_1,...,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,hist_first_buy,hist_last_buy
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,245,2,3.220431,...,-0.010108,0.010811,0.000004,5.125630,242,0.930769,2219,2462,26,269
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,396,1,4.241790,...,-0.010012,0.010667,0.000010,3.740298,390,1.114286,2244,2634,5,395
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,549,3,5.824829,...,-0.009867,-0.001998,0.000001,5.659705,412,9.581395,2217,2629,163,575
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142456,0,153,3,1.638874,...,-0.010108,0.010811,0.000007,5.736835,154,2.000000,2216,2371,25,180
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159790,0,92,4,0.741381,...,-0.010088,0.010959,0.000019,3.272112,108,0.812030,2216,2325,11,119


In [67]:
train.to_csv('mod_train.csv')

In [70]:
# Taking Reference from Other Kernels
def aggregate_transaction_new(trans, prefix):  
        
    agg_func = {
        'purchase_amount' : ['sum','max','min','mean','var','skew'],
        'installments' : ['sum','max','mean','var','skew'],
        'purchase_date' : ['max','min'],
        'month_lag' : ['max','min','mean','var','skew'],
        'month_diff' : ['max','min','mean','var','skew'],
        'weekend' : ['sum', 'mean'],
        'weekday' : ['sum', 'mean'],
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum','mean', 'max','min'],
        'card_id' : ['size','count'],
        'month': ['nunique', 'mean', 'min', 'max'],
        'hour': ['nunique', 'mean', 'min', 'max'],
        'weekofyear': ['nunique', 'mean', 'min', 'max'],
        'day': ['nunique', 'mean', 'min', 'max'],
        'subsector_id': ['nunique'],
        'merchant_category_id' : ['nunique'],
        'price' :['sum','mean','max','min','var'],
        'duration' : ['mean','min','max','var','skew'],
        'amount_month_ratio':['mean','min','max','var','skew']
    }
    
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() 
                           for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    
    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    
    return agg_trans

In [71]:
# Now extract the data from the new transactions
new_transactions = reduce_mem_usage(pd.read_csv('azureml://subscriptions/a15355bc-32e5-4151-bb4c-a2ecff532aa1/resourcegroups/ML/workspaces/MLL/datastores/workspaceblobstore/paths/UI/2024-03-25_041009_UTC/new_merchant_transactions.csv'))

Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [73]:
#impute missing values
new_transactions['category_2'] = new_transactions['category_2'].fillna(1.0,inplace=True)
new_transactions['category_3'] = new_transactions['category_3'].fillna('A',inplace=True)
new_transactions['merchant_id'] = new_transactions['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
new_transactions['installments'].replace(-1, np.nan,inplace=True)
new_transactions['installments'].replace(999, np.nan,inplace=True)
new_transactions['purchase_amount'] = new_transactions['purchase_amount'].apply(lambda x: min(x, 0.8))

#Feature Engineering - Adding new features inspired by Chau's first kernel
new_transactions['authorized_flag'] = new_transactions['authorized_flag'].map({'Y': 1, 'N': 0})
new_transactions['category_1'] = new_transactions['category_1'].map({'Y': 1, 'N': 0})
new_transactions['category_3'] = new_transactions['category_3'].map({'A':0, 'B':1, 'C':2}) 

new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])
new_transactions['month'] = new_transactions['purchase_date'].dt.month
new_transactions['weekofyear'] = new_transactions['purchase_date'].dt.isocalendar().week
new_transactions['day'] = new_transactions['purchase_date'].dt.day
new_transactions['weekday'] = new_transactions.purchase_date.dt.weekday
new_transactions['weekend'] = (new_transactions.purchase_date.dt.weekday >=5).astype(int)
new_transactions['hour'] = new_transactions['purchase_date'].dt.hour 
new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days)//30
new_transactions['month_diff'] += new_transactions['month_lag']

gc.collect()

# additional features
new_transactions['duration'] = new_transactions['purchase_amount']*new_transactions['month_diff']
new_transactions['amount_month_ratio'] = new_transactions['purchase_amount']/new_transactions['month_diff']
new_transactions['price'] = new_transactions['purchase_amount'] / new_transactions['installments']

aggs = {
        'mean': ['mean'],
    }

for col in ['category_2','category_3']:
    new_transactions[col+'_mean'] = new_transactions['purchase_amount'].groupby(new_transactions[col]).agg('mean')
    new_transactions[col+'_max'] = new_transactions['purchase_amount'].groupby(new_transactions[col]).agg('max')
    new_transactions[col+'_min'] = new_transactions['purchase_amount'].groupby(new_transactions[col]).agg('min')
    new_transactions[col+'_var'] = new_transactions['purchase_amount'].groupby(new_transactions[col]).agg('var')
    aggs[col+'_mean'] = ['mean']

gc.collect()

0

In [74]:
merge_new = aggregate_transaction_new(new_transactions, prefix='new_')
del new_transactions
gc.collect()

train = pd.merge(train, merge_new, on='card_id',how='left')
test = pd.merge(test, merge_new, on='card_id',how='left')
del merge_new

gc.collect()

[Handler.handle()] Failed to read or parse request from socket: Expecting value: line 1 column 1 (char 0)
[Handler.handle()] Failed to handle request due to exception class=JSONDecodeError, message=Expecting value: line 1 column 1 (char 0)
[Handler.handle()] Failed to read or parse request from socket: 'utf-8' codec can't decode byte 0xe4 in position 4: invalid continuation byte
[Handler.handle()] Failed to handle request due to exception class=UnicodeDecodeError, message='utf-8' codec can't decode byte 0xe4 in position 4: invalid continuation byte
[Handler.handle()] Failed to read or parse request from socket: 'utf-8' codec can't decode byte 0xe4 in position 4: invalid continuation byte
[Handler.handle()] Failed to handle request due to exception class=UnicodeDecodeError, message='utf-8' codec can't decode byte 0xe4 in position 4: invalid continuation byte
[Handler.handle()] Failed to read or parse request from socket: 'utf-8' codec can't decode byte 0x86 in position 16: invalid start

0

In [75]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,days,quarter,days_feature_1,...,new_duration_mean,new_duration_min,new_duration_max,new_duration_var,new_duration_skew,new_amount_month_ratio_mean,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,245,2,3.220431,...,-42.437298,-53.259766,-21.618408,100.250722,0.903063,-0.007813,-0.009926,-0.004002,3.396540e-06,0.891605
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,396,1,4.241790,...,-54.326416,-55.444336,-51.922852,1.755903,1.539016,-0.009700,-0.009857,-0.009482,2.308004e-08,0.808045
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,549,3,5.824829,...,-51.114258,-51.114258,-51.114258,NaN,NaN,-0.009592,-0.009592,-0.009592,NaN,NaN
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142456,0,153,3,1.638874,...,-49.207729,-54.343750,-41.950195,23.783940,0.804093,-0.008986,-0.009924,-0.007661,7.931529e-07,0.804093
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159790,0,92,4,0.741381,...,-40.852722,-54.705078,33.368652,272.900488,2.895793,-0.007500,-0.009990,0.006094,9.201419e-06,2.878926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,0.010479,0.008752,0.011428,-2.740234,0,153,3,1.603346,...,-51.308594,-51.308594,-51.308594,NaN,NaN,-0.009370,-0.009370,-0.009370,NaN,NaN
201913,2015-10-01,C_ID_1314773c0b,0.010479,0.011385,0.011428,0.312988,0,854,4,8.949397,...,-55.066406,-55.066406,-55.066406,NaN,NaN,-0.010056,-0.010056,-0.010056,NaN,NaN
201914,2017-08-01,C_ID_7666735b3d,0.010712,0.014166,0.010283,0.093506,0,184,3,1.970933,...,-17.894267,-49.104492,8.131409,839.146756,-0.779589,-0.003254,-0.008967,0.001526,2.817932e-05,-0.766576
201915,2016-07-01,C_ID_73f5a0efd0,0.010479,0.008752,0.011428,-4.675781,0,580,3,6.078045,...,-8.215098,-56.291016,62.400000,4155.786372,0.608166,-0.001397,-0.009252,0.010256,1.131874e-04,0.608140


Bad pipe message: %s [b"\x8d}K\xeat7\xbc.\xdbxy\xa2\xf6$t\x05\x8a\xfb g\x11\xcb(\tT\xee\xa5`\xf0NfJk\x0bW\x86\x18\xe1\xd7\x07=\xb3\xf8\x91\xe3\x86=u6\x08'\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t12", b'0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08']
Bad pipe message: %s [b'\x08\x08\t\x08\n\x08']
Bad pipe message: %s [b'\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 u\xe8\xfbh\x1a\xa8N`\xaa:\x16Z\x00l\xbad9\xf3\x18\x91`-']
Bad pipe message: %s [b'\xa83\x91U\x9a\x94\xcc\xc2\x8a\xfc\x16\xae\x97\xe7\xebf\xb2\xa3\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x001\x000\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99\x00\x98\x00\x97\x00\x96\x00\x07\xc0\x11\xc0

In [76]:
#Feature Engineering - Adding new features inspired by Chau's first kernel
train['new_purchase_date_max'] = pd.to_datetime(train['new_purchase_date_max'])
train['new_purchase_date_min'] = pd.to_datetime(train['new_purchase_date_min'])
train['new_purchase_date_diff'] = (train['new_purchase_date_max'] - train['new_purchase_date_min']).dt.days
train['new_purchase_date_average'] = train['new_purchase_date_diff']/train['new_card_id_size']
train['new_purchase_date_uptonow'] = (datetime.datetime.today() - train['new_purchase_date_max']).dt.days
train['new_purchase_date_uptomin'] = (datetime.datetime.today() - train['new_purchase_date_min']).dt.days
train['new_first_buy'] = (train['new_purchase_date_min'] - train['first_active_month']).dt.days
train['new_last_buy'] = (train['new_purchase_date_max'] - train['first_active_month']).dt.days
for feature in ['new_purchase_date_max','new_purchase_date_min']:
    train[feature] = train[feature].astype(np.int64) * 1e-9

#Feature Engineering - Adding new features inspired by Chau's first kernel
test['new_purchase_date_max'] = pd.to_datetime(test['new_purchase_date_max'])
test['new_purchase_date_min'] = pd.to_datetime(test['new_purchase_date_min'])
test['new_purchase_date_diff'] = (test['new_purchase_date_max'] - test['new_purchase_date_min']).dt.days
test['new_purchase_date_average'] = test['new_purchase_date_diff']/test['new_card_id_size']
test['new_purchase_date_uptonow'] = (datetime.datetime.today() - test['new_purchase_date_max']).dt.days
test['new_purchase_date_uptomin'] = (datetime.datetime.today() - test['new_purchase_date_min']).dt.days
test['new_first_buy'] = (test['new_purchase_date_min'] - test['first_active_month']).dt.days
test['new_last_buy'] = (test['new_purchase_date_max'] - test['first_active_month']).dt.days

for feature in ['new_purchase_date_max','new_purchase_date_min']:
    test[feature] = test[feature].astype(np.int64) * 1e-9
    
gc.collect()

105

In [77]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,days,quarter,days_feature_1,...,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,new_purchase_date_uptomin,new_first_buy,new_last_buy
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,245,2,3.220431,...,-0.009926,-0.004002,3.396540e-06,0.891605,54.0,2.347826,2156.0,2211.0,277.0,332.0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,396,1,4.241790,...,-0.009857,-0.009482,2.308004e-08,0.808045,56.0,9.333333,2186.0,2243.0,396.0,453.0
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,549,3,5.824829,...,-0.009592,-0.009592,NaN,NaN,0.0,0.000000,2157.0,2157.0,635.0,635.0
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142456,0,153,3,1.638874,...,-0.009924,-0.007661,7.931529e-07,0.804093,41.0,5.857143,2167.0,2209.0,187.0,229.0
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159790,0,92,4,0.741381,...,-0.009990,0.006094,9.201419e-06,2.878926,57.0,1.583333,2157.0,2214.0,121.0,178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,0.010479,0.008752,0.011428,-2.740234,0,153,3,1.603346,...,-0.009370,-0.009370,NaN,NaN,0.0,0.000000,2205.0,2205.0,191.0,191.0
201913,2015-10-01,C_ID_1314773c0b,0.010479,0.011385,0.011428,0.312988,0,854,4,8.949397,...,-0.010056,-0.010056,NaN,NaN,0.0,0.000000,2196.0,2196.0,901.0,901.0
201914,2017-08-01,C_ID_7666735b3d,0.010712,0.014166,0.010283,0.093506,0,184,3,1.970933,...,-0.008967,0.001526,2.817932e-05,-0.766576,5.0,1.666667,2188.0,2194.0,233.0,239.0
201915,2016-07-01,C_ID_73f5a0efd0,0.010479,0.008752,0.011428,-4.675781,0,580,3,6.078045,...,-0.009252,0.010256,1.131874e-04,0.608140,49.0,9.800000,2279.0,2329.0,494.0,544.0


In [78]:
#NEW Features referred from https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending
train['card_id_total'] = train['new_card_id_size']+train['hist_card_id_size']
train['card_id_cnt_total'] = train['new_card_id_count']+train['hist_card_id_count']
train['card_id_cnt_ratio'] = train['new_card_id_count']/train['hist_card_id_count']
train['purchase_amount_total'] = train['new_purchase_amount_sum']+train['hist_purchase_amount_sum']
train['purchase_amount_mean'] = train['new_purchase_amount_mean']+train['hist_purchase_amount_mean']
train['purchase_amount_max'] = train['new_purchase_amount_max']+train['hist_purchase_amount_max']
train['purchase_amount_min'] = train['new_purchase_amount_min']+train['hist_purchase_amount_min']
train['purchase_amount_ratio'] = train['new_purchase_amount_sum']/train['hist_purchase_amount_sum']
train['month_diff_mean'] = train['new_month_diff_mean']+train['hist_month_diff_mean']
train['month_diff_ratio'] = train['new_month_diff_mean']/train['hist_month_diff_mean']
train['month_lag_mean'] = train['new_month_lag_mean']+train['hist_month_lag_mean']
train['month_lag_max'] = train['new_month_lag_max']+train['hist_month_lag_max']
train['month_lag_min'] = train['new_month_lag_min']+train['hist_month_lag_min']
train['category_1_mean'] = train['new_category_1_mean']+train['hist_category_1_mean']
train['installments_total'] = train['new_installments_sum']+train['hist_installments_sum']
train['installments_mean'] = train['new_installments_mean']+train['hist_installments_mean']
train['installments_max'] = train['new_installments_max']+train['hist_installments_max']
train['installments_ratio'] = train['new_installments_sum']/train['hist_installments_sum']
train['price_total'] = train['purchase_amount_total'] / train['installments_total']
train['price_mean'] = train['purchase_amount_mean'] / train['installments_mean']
train['price_max'] = train['purchase_amount_max'] / train['installments_max']
train['duration_mean'] = train['new_duration_mean']+train['hist_duration_mean']
train['duration_min'] = train['new_duration_min']+train['hist_duration_min']
train['duration_max'] = train['new_duration_max']+train['hist_duration_max']
train['amount_month_ratio_mean']=train['new_amount_month_ratio_mean']+train['hist_amount_month_ratio_mean']
train['amount_month_ratio_min']=train['new_amount_month_ratio_min']+train['hist_amount_month_ratio_min']
train['amount_month_ratio_max']=train['new_amount_month_ratio_max']+train['hist_amount_month_ratio_max']
train['new_CLV'] = train['new_card_id_count'] * train['new_purchase_amount_sum'] / train['new_month_diff_mean']
train['hist_CLV'] = train['hist_card_id_count'] * train['hist_purchase_amount_sum'] / train['hist_month_diff_mean']
train['CLV_ratio'] = train['new_CLV'] / train['hist_CLV']

test['card_id_total'] = test['new_card_id_size']+test['hist_card_id_size']
test['card_id_cnt_total'] = test['new_card_id_count']+test['hist_card_id_count']
test['card_id_cnt_ratio'] = test['new_card_id_count']/test['hist_card_id_count']
test['purchase_amount_total'] = test['new_purchase_amount_sum']+test['hist_purchase_amount_sum']
test['purchase_amount_mean'] = test['new_purchase_amount_mean']+test['hist_purchase_amount_mean']
test['purchase_amount_max'] = test['new_purchase_amount_max']+test['hist_purchase_amount_max']
test['purchase_amount_min'] = test['new_purchase_amount_min']+test['hist_purchase_amount_min']
test['purchase_amount_ratio'] = test['new_purchase_amount_sum']/test['hist_purchase_amount_sum']
test['month_diff_mean'] = test['new_month_diff_mean']+test['hist_month_diff_mean']
test['month_diff_ratio'] = test['new_month_diff_mean']/test['hist_month_diff_mean']
test['month_lag_mean'] = test['new_month_lag_mean']+test['hist_month_lag_mean']
test['month_lag_max'] = test['new_month_lag_max']+test['hist_month_lag_max']
test['month_lag_min'] = test['new_month_lag_min']+test['hist_month_lag_min']
test['category_1_mean'] = test['new_category_1_mean']+test['hist_category_1_mean']
test['installments_total'] = test['new_installments_sum']+test['hist_installments_sum']
test['installments_mean'] = test['new_installments_mean']+test['hist_installments_mean']
test['installments_max'] = test['new_installments_max']+test['hist_installments_max']
test['installments_ratio'] = test['new_installments_sum']/test['hist_installments_sum']
test['price_total'] = test['purchase_amount_total'] / test['installments_total']
test['price_mean'] = test['purchase_amount_mean'] / test['installments_mean']
test['price_max'] = test['purchase_amount_max'] / test['installments_max']
test['duration_mean'] = test['new_duration_mean']+test['hist_duration_mean']
test['duration_min'] = test['new_duration_min']+test['hist_duration_min']
test['duration_max'] = test['new_duration_max']+test['hist_duration_max']
test['amount_month_ratio_mean']=test['new_amount_month_ratio_mean']+test['hist_amount_month_ratio_mean']
test['amount_month_ratio_min']=test['new_amount_month_ratio_min']+test['hist_amount_month_ratio_min']
test['amount_month_ratio_max']=test['new_amount_month_ratio_max']+test['hist_amount_month_ratio_max']
test['new_CLV'] = test['new_card_id_count'] * test['new_purchase_amount_sum'] / test['new_month_diff_mean']
test['hist_CLV'] = test['hist_card_id_count'] * test['hist_purchase_amount_sum'] / test['hist_month_diff_mean']
test['CLV_ratio'] = test['new_CLV'] / test['hist_CLV']

In [79]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,days,quarter,days_feature_1,...,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,245,2,3.220431,...,0.503857,-89.998522,-107.974962,37.581592,-0.016532,-0.020034,0.006809,-4.133346,-589.360250,0.007013
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,396,1,4.241790,...,0.008940,-100.408192,-111.124322,8.077148,-0.017919,-0.019869,0.001185,-0.349214,-1006.710678,0.000347
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,549,3,5.824829,...,-inf,-101.243774,-105.144483,-61.761112,-0.018770,-0.019458,-0.011590,-0.009592,-16.975293,0.000565
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142456,0,153,3,1.638874,...,0.058276,-97.272510,-109.170141,17.249805,-0.017807,-0.020032,0.003150,-0.440318,-52.288836,0.008421
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159790,0,92,4,0.741381,...,0.089352,-82.625484,-109.920656,92.568652,-0.015162,-0.020077,0.017053,-9.715684,-135.611337,0.071644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,0.010479,0.008752,0.011428,-2.740234,0,153,3,1.603346,...,-1.265268,-103.744816,-106.246180,-93.629861,-0.018963,-0.019535,-0.017098,-0.009370,-20.298448,0.000462
201913,2015-10-01,C_ID_1314773c0b,0.010479,0.011385,0.011428,0.312988,0,854,4,8.949397,...,-1.318394,-107.705475,-110.116303,-97.561139,-0.019669,-0.020109,-0.017816,-0.010056,-21.234423,0.000474
201914,2017-08-01,C_ID_7666735b3d,0.010712,0.014166,0.010283,0.093506,0,184,3,1.970933,...,0.075949,-52.785647,-103.710717,67.331409,-0.009640,-0.018939,0.012485,-0.029482,-48.371005,0.000609
201915,2016-07-01,C_ID_73f5a0efd0,0.010479,0.008752,0.011428,-4.675781,0,580,3,6.078045,...,inf,-61.857946,-113.377754,14.857411,-0.010222,-0.018635,0.002442,-0.034431,-5.965927,0.005771


In [80]:
train.head(10)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,outliers,days,quarter,days_feature_1,...,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820312,0,245,2,3.220431,...,0.503857,-89.998522,-107.974962,37.581592,-0.016532,-0.020034,0.006809,-4.133346,-589.360250,0.007013
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392822,0,396,1,4.241790,...,0.008940,-100.408192,-111.124322,8.077148,-0.017919,-0.019869,0.001185,-0.349214,-1006.710678,0.000347
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.687988,0,549,3,5.824829,...,-inf,-101.243774,-105.144483,-61.761112,-0.018770,-0.019458,-0.011590,-0.009592,-16.975293,0.000565
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142456,0,153,3,1.638874,...,0.058276,-97.272510,-109.170141,17.249805,-0.017807,-0.020032,0.003150,-0.440318,-52.288836,0.008421
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159790,0,92,4,0.741381,...,0.089352,-82.625484,-109.920656,92.568652,-0.015162,-0.020077,0.017053,-9.715684,-135.611337,0.071644
5,2016-09-01,C_ID_0894217f2f,0.010712,0.008752,0.010283,0.871582,0,518,3,5.548604,...,0.002242,-62.159290,-108.020416,2.723974,-0.009474,-0.016464,0.000415,-0.069123,-5.612538,0.012316
6,2016-12-01,C_ID_7e63323c00,0.010479,0.008752,0.011428,0.230103,0,427,4,4.474699,...,-inf,-106.075602,-109.870449,-91.700667,-0.019377,-0.020196,-0.016746,-0.243415,-651.684454,0.000374
7,2017-09-01,C_ID_dfa21fc124,0.010479,0.008752,0.011428,2.136719,0,153,3,1.603346,...,-inf,-99.830481,-109.976962,-57.168806,-0.017349,-0.019171,-0.009917,-0.082821,-3.943465,0.021002
8,2017-08-01,C_ID_fe0fdac8ea,0.010610,0.011385,0.010283,-0.065430,0,184,3,1.952220,...,-0.277330,-89.699107,-105.729249,-61.567260,-0.016380,-0.019308,-0.011243,-0.034140,-1.765211,0.019341
9,2016-08-01,C_ID_bf62c0b49d,0.010610,0.008752,0.010283,0.300049,0,549,3,5.824829,...,-1.037351,-103.125103,-109.890050,-76.392622,-0.018864,-0.020193,-0.014021,-0.086762,-117.787981,0.000737


In [81]:
train.to_csv('mod_train_newtrans.csv')

In [82]:
test.to_csv('mod_test_newtrans.csv')

In [83]:
test

,first_active_month,card_id,feature_1,feature_2,feature_3,days,quarter,days_feature_1,days_feature_1_ratio,days_feature_2,...,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio
0,2017-04-01,C_ID_0ab67a22ab,0.010479,0.014166,0.011428,306.0,2.0,3.206693,0.000034,4.334919,...,-0.009842,-90.240887,-110.699188,-11.219498,-0.015732,-0.019417,-0.001942,-0.070467,-36.495404,0.001931
1,2017-01-01,C_ID_130fd0cbdd,0.010610,0.014166,0.010283,396.0,1.0,4.201516,0.000027,5.609895,...,-0.026790,-95.492343,-108.972690,-13.877232,-0.017438,-0.019900,-0.002534,-0.722960,-51.792541,0.013959
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011385,0.011428,184.0,3.0,2.418609,0.000071,2.094798,...,0.080000,12.580680,-93.324805,118.400000,0.002297,-0.017043,0.021622,0.002038,0.302176,0.006743
3,2017-12-01,C_ID_d27d835a9f,0.010610,0.011385,0.010283,62.0,4.0,0.657813,0.000171,0.705856,...,-0.020002,-81.424866,-103.877963,-26.642431,-0.014888,-0.018970,-0.004865,-0.777319,-4.810251,0.161596
4,2015-12-01,C_ID_2b5e3df5c2,0.013145,0.011385,0.011428,793.0,4.0,10.423680,0.000017,9.028126,...,0.266667,-48.417975,-107.363571,118.400000,-0.008857,-0.019606,0.021622,-0.114717,-68.576505,0.001673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10-01,C_ID_7a239d2eda,0.010610,0.008752,0.010283,123.0,4.0,1.305016,0.000086,1.076511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-48.872857,NaN
123619,2017-09-01,C_ID_75ace375ae,0.010479,0.011385,0.011428,153.0,3.0,1.603346,0.000068,1.741870,...,-inf,-101.131269,-109.307390,-82.549402,-0.018468,-0.019961,-0.015075,-0.143687,-0.464896,0.309073
123620,2016-09-01,C_ID_21d56d950c,0.013145,0.011385,0.011428,518.0,3.0,6.808911,0.000025,5.897313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.919737,NaN
123621,2017-06-01,C_ID_6c46fc5a9d,0.010610,0.011385,0.010283,245.0,2.0,2.599423,0.000043,2.789270,...,-0.076852,-98.306267,-112.780179,-47.955784,-0.016175,-0.018582,-0.007882,-0.290753,-31.130454,0.009340


In [6]:
# Can start importing from here
train = pd.read_csv('mod_train_newtrans.csv', index_col=[0])
test = pd.read_csv('mod_test_newtrans.csv', index_col=[0])

In [7]:
train = train.drop(['card_id', 'first_active_month'], axis = 1)
test = test.drop(['card_id', 'first_active_month'], axis = 1)

In [8]:
# Get the X and Y
df_train_columns = [c for c in train.columns if c not in ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size']] 
target = train['target']
del train['target']

In [9]:
target

0        -0.8203
1         0.3928
2         0.6880
3         0.1425
4        -0.1598
           ...  
201912   -2.7400
201913    0.3130
201914    0.0935
201915   -4.6760
201916   -1.8590
Name: target, Length: 201917, dtype: float64

In [10]:
train.shape

(201917, 192)

In [11]:
train[df_train_columns][:50000]

,feature_1,feature_2,feature_3,days,quarter,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,...,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio
0,0.013145,0.008752,0.011428,245,2,3.220431,0.000054,2.144270,0.000036,2.799743,...,0.503857,-89.998522,-107.974962,37.581592,-0.016532,-0.020034,0.006809,-4.133346,-589.360250,0.007013
1,0.010712,0.011385,0.010283,396,1,4.241790,0.000027,4.508370,0.000029,4.072003,...,0.008940,-100.408192,-111.124322,8.077148,-0.017919,-0.019869,0.001185,-0.349214,-1006.710678,0.000347
2,0.010610,0.008752,0.010283,549,3,5.824829,0.000019,4.804915,0.000016,5.645276,...,-inf,-101.243774,-105.144483,-61.761112,-0.018770,-0.019458,-0.011590,-0.009592,-16.975293,0.000565
3,0.010712,0.014166,0.010283,153,3,1.638874,0.000070,2.167460,0.000093,1.573274,...,0.058276,-97.272510,-109.170141,17.249805,-0.017807,-0.020032,0.003150,-0.440318,-52.288836,0.008421
4,0.008058,0.014166,0.010283,92,4,0.741381,0.000088,1.303309,0.000154,0.946021,...,0.089352,-82.625484,-109.920656,92.568652,-0.015162,-0.020077,0.017053,-9.715684,-135.611337,0.071644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.010479,0.008752,0.011428,427,4,4.474699,0.000025,3.737156,0.000020,4.879551,...,-1.321949,-107.008106,-109.459484,-99.146193,-0.019024,-0.019459,-0.017626,-0.084590,-32.377799,0.002613
49996,0.013145,0.008752,0.011428,580,3,7.623877,0.000023,5.076230,0.000015,6.627962,...,-0.996818,-102.134596,-106.971312,-73.107816,-0.018863,-0.019865,-0.013592,-0.151531,-353.487664,0.000429
49997,0.010712,0.011385,0.010283,92,4,0.985466,0.000116,1.047399,0.000124,0.946021,...,-0.000934,-85.674798,-104.518650,-0.483692,-0.015726,-0.019343,-0.000088,-2.552364,-33.155082,0.076983
49998,0.010479,0.008752,0.011428,276,2,2.892311,0.000038,2.415585,0.000032,3.153996,...,-0.244143,-100.659465,-109.389735,-18.066578,-0.018421,-0.020079,-0.003299,-0.039670,-97.377273,0.000407


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(train[df_train_columns][:100000], target[:100000], test_size=0.2, random_state=42)

del X_train['target']
del X_train['first_active_month']
del X_train['card_id']

del X_test['target']
del X_test['first_active_month']
del X_test['card_id']

# Toggle

# Standardize Features
from sklearn.preprocessing import StandardScaler

# Use StandardScaler to scale the training and validation data
scaler = StandardScaler()

#Fit the StandardScaler to the training data
scaler.fit(X_train)

# transform both the training and validation data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# The baseline prediction is average of the training target labels
baseline_preds = np.mean(Y_train)
baseline_errors = [baseline_preds] * len(Y_test)
# Calculate the baseline errors and display average baseline error
mae = mean_absolute_error(Y_test, baseline_errors)
rmse = mean_squared_error(Y_test, baseline_errors, squared=False)

print('Model Performance')
print('MAE:', round(mae, 2))
print('RMSE:', round(rmse, 2))

Model Performance
MAE: 1.58
RMSE: 3.76


In [15]:
from sklearn.impute import SimpleImputer

# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)

In [16]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators=50, random_state=42)

# Train the model on training data
rf.fit(X_train, Y_train)

RandomForestRegressor(n_estimators=50, random_state=42)

In [17]:
def evaluate_model(model, X_test, Y_test):
    predictions = model.predict(X_test)
    
    mae = mean_absolute_error(Y_test, predictions)
    rmse = mean_squared_error(Y_test, predictions, squared=False)
    
    print('Model Performance')
    print('MAE:', round(mae, 2))
    print('RMSE:', round(rmse, 2))

In [19]:
# Assuming you have trained your random forest model (rf) and have the test data (X_test and Y_test)
evaluate_model(rf, X_test, Y_test)

Model Performance
MAE: 1.72
RMSE: 3.71


In [28]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [31]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 150, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [30, 43, 56, 70, 83, 96, 110, 123, 136, 150], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [32]:
# Use the random grid to search for best hyperparameters

# First create the base model to tune
rf = RandomForestRegressor()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
random_grid = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42)

# Fit the random search model
random_grid.fit(X_train, Y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=30; total time=  15.7s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=30; total time=  15.4s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=30; total time=  15.7s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=56; total time=  40.0s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=56; total time=  40.8s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=56; total time=  40.8s
[CV] END bootstrap=False, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=56; tot

Bad pipe message: %s [b'N\xf1\xd1V\xd5\xe5\xed\xf4\xcd\xe1\xd0\xc3']
Bad pipe message: %s [b'\x9b\xf4\xaa$M\xfe\x0f\x05']
Bad pipe message: %s [b"\x1aB\x03N\xef0\x1a\x0c\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0", b'3\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\xc0\x11\xc0\x08\xc0']
Bad pipe message: %s [b'\x16\x00\x13\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c']
Bad pipe message: %s [b'k\x9f\xc9qI\xaf\xd4\x1f\xaa\x8a\xaf\x13\xbcD\xbbX\xf3\xc1\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x001\x000\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99\x00\x98\x00\x97\x00\x96\x00\x07\xc0\x11\xc0\x07\xc0\x0c\xc0\x02\x00\x05\x00\x04\x00\xff\x02\x01\x00\x00C\x00\x00\x00\x0

In [ ]:
import pickle
 
filename = 'model_random_grid_Mar.pkl'
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
random_grid.best_params_

In [ ]:
# Assuming you have trained your random forest model (rf) and have the test data (X_test and Y_test)
evaluate_model(random_grid, X_test.iloc[:,top_10_indexes], Y_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [5,10,15],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2,3],
    'min_samples_split': [4,5,6],
    'n_estimators': [175,200,225]
}
# Create a based model  
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the random search model
grid_search.fit(X_train.iloc[:,top_10_indexes], Y_train)

In [ ]:
# Assuming you have trained your random forest model (rf) and have the test data (X_test and Y_test)
evaluate_model(grid_search, X_test.iloc[:,top_10_indexes], Y_test)

In [ ]:
grid_search.best_params_

In [47]:
import pickle
 
filename = 'grid_search_model.pkl'
pickle.dump(grid_search, open(filename, 'wb'))

[CV] END bootstrap=True, max_depth=15, max_features=sqrt, min_samples_leaf=3, min_samples_split=6, n_estimators=200; total time= 1.1min
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=175; total time=  20.8s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=4, n_estimators=225; total time=  27.1s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=175; total time=  21.5s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=  23.3s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=225; total time=  27.0s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=6, n_estimators=200; total time=  24.3s
[CV] END bootstrap=True, max_depth=5, max_features=sqr